In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import re

In [2]:
months_dtype = CategoricalDtype(categories=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], ordered=True)

### Load in 2009 to 2014 As Is

In [3]:
df_2009_2014 = pd.read_csv("finished-csvs/finished-2009-2014.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2009_2014.head()

,Institution,Year,Month,Use of Force,Inmate Count
0,CAL,2009,Jan,26,4272
1,CEN,2009,Jan,13,4834
2,CMC,2009,Jan,7,6605
3,CMF,2009,Jan,19,2831
4,MCSP,2009,Jan,19,3880


### Load in Population for 2015 to 2019

In [4]:
df_2015_2019_POP = pd.read_csv("finished-csvs/finished-2015-2019-POPULATION.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2015_2019_POP.head()

,Institution,Year,Month,Inmate Count
0,ASP,2015,Jan,4064
1,CAL,2015,Jan,3747
2,CCC,2015,Jan,4462
3,CCI,2015,Jan,4189
4,CEN,2015,Jan,3215


### Load in UOF for 2015 to 2019

(Note, this sheet technically will load in UOF cases up to 2020, but we'll pair those later)

In [5]:
df_2015_2020_UOF = pd.read_csv("finished-csvs/finished-2015-2020-UOF.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2015_2020_UOF.head()

,Institution,Year,Month,Use of Force
0,ASP,2015,Jan,12
1,CAC,2015,Jan,1
2,CAL,2015,Jan,16
3,CCC,2015,Jan,15
4,CCI,2015,Jan,7


### Merge Pop and UOF from 2015 to 2019

In [6]:
df_2015_2019 = df_2015_2019_POP.merge(df_2015_2020_UOF, on=['Institution','Year','Month'], how='inner')
df_2015_2019.head()

,Institution,Year,Month,Inmate Count,Use of Force
0,ASP,2015,Jan,4064,12
1,CAL,2015,Jan,3747,16
2,CCC,2015,Jan,4462,15
3,CCI,2015,Jan,4189,7
4,CEN,2015,Jan,3215,10


### Load in UOF from 2020 to 2021

(Note: UOF cases were omitted from these reports for 2020, so we'll use the COMPSTAT report data for that year)

In [7]:
df_2020_2021_missing_UOF_2020 = pd.read_csv("finished-csvs/finished-2020-2021.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2020_2021_missing_UOF_2020.head()

,Institution,Year,Month,Use of Force,Inmate Count
0,CAC,2020,Jan,-,2140
1,CCI,2020,Jan,-,3700
2,COR,2020,Jan,-,3042
3,HDSP,2020,Jan,-,3161
4,KVSP,2020,Jan,-,3553


### Filter just 2021

In [8]:
df_2021 = df_2020_2021_missing_UOF_2020[df_2020_2021_missing_UOF_2020['Year'] == 2021]
df_2021

,Institution,Year,Month,Use of Force,Inmate Count
10,CAC,2021,Jan,6,1991
11,CCI,2021,Jan,18,2929
12,COR,2021,Jan,9,2917
13,HDSP,2021,Jan,21,3296
14,KVSP,2021,Jan,45,3629
...,...,...,...,...,...
235,LAC,2021,Dec,41,2771
236,PBSP,2021,Dec,11,1985
237,SAC,2021,Dec,44,1908
238,SATF,2021,Dec,19,4958


### Merge 2020 UOF with missing data from 2015 to 2020 CSV file

In [9]:
df_2020 = df_2020_2021_missing_UOF_2020.merge(df_2015_2020_UOF, how='inner', on=['Institution','Year','Month'])
df_2020.drop('Use of Force_x', axis=1)
df_2020.rename(columns={'Use of Force_y':'Use of Force'}, inplace=True)
df_2020 = df_2020[['Institution','Year','Month', 'Use of Force', 'Inmate Count']]
df_2020

,Institution,Year,Month,Use of Force,Inmate Count
0,CAC,2020,Jan,6,2140
1,CCI,2020,Jan,41,3700
2,COR,2020,Jan,26,3042
3,HDSP,2020,Jan,27,3161
4,KVSP,2020,Jan,37,3553
...,...,...,...,...,...
115,LAC,2020,Dec,18,2704
116,PBSP,2020,Dec,5,2209
117,SAC,2020,Dec,48,2227
118,SATF,2020,Dec,22,4315


### Load in 2022 to 2023 pouplation data

In [10]:
df_2022_2023_population = pd.read_csv("finished-csvs/finished-2022-2023-POPULATION.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2022_2023_population.head()

,Institution,Year,Month,Inmate Count
0,CAC,2022,Jun,1963
1,CCI,2022,Jun,3051
2,COR,2022,Jun,3466
3,HDSP,2022,Jun,1968
4,KVSP,2022,Jun,3045


### Load in 2022 to 2023 UOF data

In [11]:
df_2022_2023_UOF = pd.read_csv("finished-csvs/finished-2022-2023-UOF.csv", encoding="utf-8", dtype={'Use of Force':'Int64', 'Month':months_dtype})
df_2022_2023_UOF.head()

,Institution,Month,Use of Force,Year
0,ASP,Oct,5,2022
1,CAC,Oct,12,2022
2,CAL,Oct,4,2022
3,CCC,Oct,1,2022
4,CCI,Oct,41,2022


### Merge 2022 to 2023 population and UOF 

In [12]:
df_2022_2023 = df_2022_2023_population.merge(df_2022_2023_UOF, how='inner', on=['Institution','Year','Month'])
df_2022_2023 = df_2022_2023[['Institution','Year','Month', 'Inmate Count', 'Use of Force']]
df_2022_2023

,Institution,Year,Month,Inmate Count,Use of Force
0,CAC,2022,Oct,1936,12
1,CCI,2022,Oct,2890,41
2,COR,2022,Oct,3323,45
3,HDSP,2022,Oct,2382,9
4,KVSP,2022,Oct,2611,22
...,...,...,...,...,...
85,LAC,2023,Jun,2660,91
86,PBSP,2023,Jun,1627,9
87,SAC,2023,Jun,1759,47
88,SATF,2023,Jun,4598,29


## Merge Everything together into one DF

In [13]:
df_final = pd.concat([df_2009_2014, df_2015_2019, df_2020, df_2021, df_2022_2023], ignore_index=True)
df_final = df_final.sort_values(by=['Year','Month','Institution'], ignore_index=True)
df_final

,Institution,Year,Month,Use of Force,Inmate Count
0,CAL,2009,Jan,26,4272
1,CCI,2009,Jan,12,5703
2,CEN,2009,Jan,13,4834
3,CIM,2009,Jan,24,5673
4,CMC,2009,Jan,7,6605
...,...,...,...,...,...
3169,LAC,2023,Jun,91,2660
3170,PBSP,2023,Jun,9,1627
3171,SAC,2023,Jun,47,1759
3172,SATF,2023,Jun,29,4598


### Filter for Only High Security Prisons

In [14]:
high_security = ['CAC', 'CCI', 'COR', 'LAC', 'SAC', 'HDSP', 'KVSP', 'PBSP', 'SATF', 'SVSP']
df_high_security = df_final[df_final['Institution'].isin(high_security)]
df_high_security

,Institution,Year,Month,Use of Force,Inmate Count
1,CCI,2009,Jan,12,5703
6,COR,2009,Jan,21,5654
8,HDSP,2009,Jan,30,4415
9,KVSP,2009,Jan,26,4789
10,LAC,2009,Jan,36,4906
...,...,...,...,...,...
3169,LAC,2023,Jun,91,2660
3170,PBSP,2023,Jun,9,1627
3171,SAC,2023,Jun,47,1759
3172,SATF,2023,Jun,29,4598


In [16]:
df_high_security.to_csv("finished-csvs/all_data_final.csv", encoding="utf-8", index=False)